**About**: We find the association between company level indicators (ratings) and state indicators against ODI.

In [ ]:
construct = 'odi'
interact = True
split = False # flag for splitting ODI into physio and psycho

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import pickle
from stargazer.stargazer import Stargazer

scaler = MinMaxScaler()

matplotlib.rc('xtick', labelsize=16)
matplotlib.rc('ytick', labelsize=16)
plt.rcParams["font.family"] = "lato"
plt.rcParams["font.weight"] = "bold"
plt.rcParams['text.color'] = 'black'
#plt.ticklabel_format(style='sci')

font = {'size'   : 18}

#matplotlib.rc('font', **font)


DATAROOT = "../data/"
PLOTROOT = "../plots/"

# params
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('xtick', labelsize=16)
matplotlib.rc('ytick', labelsize=16)
plt.ticklabel_format(style='sci', axis='y')


import pickle
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# params
from utils import read_config

config = read_config()

DATAROOT = "../data/"
PLOTROOT = "../plots/ODI/states/"

pros_cons = ['pros', 'cons']


# params
sent_based = True
text = config['TEXTS'][construct]
num_goals = len(config['COUNTS'][construct])
THRESHOLD = config['THRESHOLDS'][construct]
PRESET = config['PRESETS'][construct]
primary_goal_shorthand = config['SHORTHANDS'][construct]
final_goals = config['FINAL'][construct]
scoring = config['SCORING'][construct]
reference_text = config['REFERENCES'][construct]
num_goals = len(config['COUNTS'][construct])
sim_num = 0

metric = 'avg sim score' # or 'pro proportion' (unweighted)

In [ ]:
text

In [ ]:
def scale_var(data, var):
    scaled_data = [i[0]*100 for i in list(scaler.fit_transform(data[[var]]))]
    return scaled_data

In [ ]:
# DEPENDENT VARIABLE: first get stocks data
stocks = pd.read_csv(DATAROOT + "companies_stock.csv")
stocks = stocks.dropna(subset = ['stock_2009', 'stock_2019'])
stocks['total_stock_growth'] = stocks['stock_2019'] / stocks['stock_2009']
stocks['log_total_stock_growth'] = np.log(stocks['total_stock_growth'])
stocks

In [ ]:
stocks.columns

In [ ]:
stocks.corr()['log_total_stock_growth']

In [ ]:
stocks['log_total_stock_growth'].plot(kind = 'hist', bins = 30)

In [ ]:
review = pd.read_csv(DATAROOT + "review_us_master.csv")
review.head()

In [ ]:
# get sustainability variables

In [ ]:
construct

In [ ]:
from utils import sample_data

#onek_data = sample_data(review, review_threshold = 500)
#onek_data = sample_data(review, review_threshold = 1050)
#onek_data = sample_data(review, 900)
onek_data = sample_data(review)

sustainability = pd.read_csv(DATAROOT + config['REFERENCE_DATAS'][construct], sep = "\t")

main_datas = {}
main_datas_ = {}

for text in pros_cons:
    if sent_based:
        embed_file = "intermediate/%s_%s_sent_embedded.csv" %(construct, text) # sentence-based
    else:
        embed_file = "intermediate/%s_embedded_sim_review.csv" %(text) # original full pro or con
        
    main_datas_[text] = pd.read_csv(DATAROOT+embed_file, sep = "\t")
    #main_datas[text] = []
    #main_datas[text].append(subset_by_percentile_or_preset(main_datas_[text], 0.8, simfield = '_sim_1', preset = 0.20))


In [ ]:
# # add nokia data
# for text in ['pros']:
# #     if sent_based:
# #         embed_file = "intermediate/%s_sent_embedded.csv" %(text) # sentence-based
# #     else:
# #         embed_file = "intermediate/%s_embedded_sim_review.csv" %(text) # original full pro or con
#     if sent_based:
#         embed_file = "intermediate/nokia_data_%s_sent_embedded.csv" %(text) # sentence-based
#     else:
#         embed_file = "intermediate/%s_embedded_sim_review.csv" %(text) # original full pro or con
        
#     main_datas_[text] = main_datas_[text].append(pd.read_csv(DATAROOT+embed_file, sep = "\t")).reset_index()

    
# # add nokia data to onek_data and main_data
# onek_data = onek_data.append(pd.read_csv("../data/nokia_data.csv")).reset_index()
# review = review.append(pd.read_csv("../data/nokia_data.csv")).reset_index()    

In [ ]:
from utils import subset_by_percentile_or_preset, aggregate_sents


posted_sim = {}


for text_ in pros_cons:
    data_90 = subset_by_percentile_or_preset(main_datas_[text_], THRESHOLD, preset = PRESET, num = num_goals)
    if sent_based:
        data_90 = aggregate_sents(data_90, review, num_goals = num_goals)
        top5_by_goals = {}
        ### also save full list of companies
        company_list = {}
        posts = []
        for num in final_goals:
            posts.append(data_90[num]['company_id'])
            company = pd.DataFrame(data_90[num].groupby("company_name").size(),
                                       columns = ['%d reviews' %num]).reset_index()
                
            
            company['avg sim score'] = list(data_90[num].groupby("company_name").sum()['%d_sim_1' %num].values)
            company_total_reviews = pd.DataFrame(onek_data.groupby("company_name").size(), columns = ['total']).reset_index()
            company = company.merge(company_total_reviews, on = 'company_name')
            company['%d reviews normalized' %(num)] = company['%d reviews' %(num)] / company['total']
            company['avg sim score'] = company['avg sim score'] / company['total']
            
            
            
            
            top5 = company.sort_values(['%d reviews normalized' %(num)], ascending = False)
            top5[reference_text] = [sustainability[reference_text][num]] * len(top5)
            top5.columns = ['company', 'shortlisted reviews', 'avg sim score',
                                'total reviews', 'pro proportion', reference_text]
            top5_by_goals[num] = top5
            company_list[sustainability[reference_text][num]] = list(top5['company'].values)
            # find the z-score of each goal
            #z_score = 
        # add the composite score here
        
        posted_sim = posts

In [ ]:
from utils import aggregate_area

company_data = {}

for text_ in [text]:
    data_90 = subset_by_percentile_or_preset(main_datas_[text], THRESHOLD, preset = PRESET, num = num_goals)
    if sent_based:
        data_90 = aggregate_sents(data_90, review, num_goals = num_goals)              
    company_data[text] = aggregate_area(data_90, final_goals, onek_data,
                                      sustainability,
                                      #metric = 'pro proportion',
                                      construct = construct,
                                      area_name = 'company_name',
                                      find_pcs = False,
                                      reference_text = reference_text,
                                      scaled = True, scoring = scoring)

In [ ]:
company_data[text][1].corr()

In [ ]:
stocks.columns

In [ ]:
for num in final_goals:
    print(len(top5_by_goals[num]))
    stocks = stocks.merge(top5_by_goals[num][['company', metric]], on = 'company')
    stocks.rename({'%s' %metric : '%d %s' %(num, metric)}, axis=1, inplace=True)
    
stocks    

In [ ]:
stocks_again = pd.read_csv(DATAROOT+"companies_stock.csv")
stocks_again = stocks_again.rename({'company' : 'company_name'},  axis=1)
company_stocks = company_data[text][1].copy()
company_stocks = company_stocks.merge(stocks_again, on = "company_name")

In [ ]:
company_total_reviews = pd.DataFrame(review.groupby('company_name').size()).reset_index()
company_total_reviews.columns = ['company_name', 'total reviews']
company_stocks = company_stocks.merge(company_total_reviews, on = 'company_name')
company_stocks

In [ ]:
company_stocks[['total_posts', 'total reviews']].corr()

### actual regression

as a starting point, try predicting log_total_stock_growth with total reviews, average rating and the different ratings

In [ ]:
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

In [ ]:
def regress(data, evs, dv):
    Y = data[dv]
    X = sm.add_constant(data[evs])
    model = sm.OLS(Y,X)
    results = model.fit()
    return results

In [ ]:
# stepaic: https://stackoverflow.com/questions/22428625/does-statsmodels-or-another-python-package-offer-an-equivalent-to-rs-step-f

import itertools

def stepaic_model(train, predictorcols, target):

    AICs = {}

    for k in range(1,len(predictorcols)+1):
        for variables in itertools.combinations(predictorcols, k):
            predictors = train[list(variables)]
            predictors = sm.add_constant(predictors)
            res = sm.OLS(train[target], predictors).fit()
            AICs[variables] = 2*(k+1) - 2*res.llf
    
    min_so_far = (list(AICs.keys())[0], AICs[list(AICs.keys())[0]])
    for key in AICs:
        if AICs[key] < AICs[min_so_far[0]]:
            min_so_far = (key, AICs[key])
        
    return min_so_far   

In [ ]:
stocks.columns

In [ ]:
data = stocks
dv = 'log_total_stock_growth'
evs = ['total_posts', 
            'rating_balance', 'rating_career', 'rating_comp', 'rating_culture', 'rating_mgmt',
       'rating_overall']
rating_model = regress(stocks, evs, dv)

now add the sustaianbaility variables.

In [ ]:
selected_columns = ['total_posts',
            #'rating_balance', 'rating_career', 'rating_comp', 'rating_culture', 'rating_mgmt',
       #'rating_overall'
                   ]
selected_columns.extend(["%d %s" %(i, metric) for i in final_goals])
sustainable_model = regress(stocks, evs = selected_columns, dv = dv)

In [ ]:
dfoutput = summary_col([sustainable_model],stars=True)
print(dfoutput)

In [ ]:
# scatter plots for the stock growth
fig, ax = plt.subplots(nrows = 1, ncols = 6, figsize = (24, 4), sharey = True)
for n, num in enumerate(final_goals):
    sns.scatterplot(data = stocks, y = 'log_total_stock_growth', x = '%d %s' %(num, metric), ax = ax[n])
    corr, p = stats.pearsonr(stocks['log_total_stock_growth'], stocks['%d %s' %(num, metric)])
    ax[n].set_title("%s\n%0.2f %0.2f" %(primary_goal_shorthand[sustainability[reference_text][num]],
                                        corr, p), fontsize = 18)
    if n == 0:
        ax[n].set_ylabel("log_total_stock_growth", fontsize = 18)
    ax[n].set_xlabel(metric, fontsize = 18)
    

In [ ]:
# todo: predict company ratings from sustainability scores

In [ ]:
stocks.columns

In [ ]:
evs = ['%d avg sim score' %i for i in final_goals]
evs = evs + ['total_posts']#, 'composite']

In [ ]:
dvs = ['rating_career', 'rating_comp', 'rating_culture', 'rating_mgmt',
       'rating_overall']

In [ ]:
rating_models = []
for dv in dvs:
    rating_models.append(regress(stocks, evs = evs, dv = dv))

In [ ]:
dvs

In [ ]:
Stargazer(rating_models)

In [ ]:
evs = ['%d avg sim score' %i for i in final_goals]
evs = evs + ['total_posts', 'composite']

dvs = ['rating_balance',
       'rating_career', 'rating_comp', 'rating_culture', 'rating_mgmt',
       'rating_overall']

In [ ]:
# with the differently aggregated data

rating_models = []
for dv in dvs:
    company_stocks[dv + '_scaled'] = scale_var(company_stocks, dv)
    rating_models.append(regress(company_stocks, evs = evs, dv = dv + '_scaled'))

In [ ]:
Stargazer(rating_models)

In [ ]:
# with stepaic

rating_models = []
for dv in dvs:
    stepaic_evs, _ = stepaic_model(company_stocks, evs, dv + '_scaled')
    rating_models.append(regress(company_stocks, evs = list(stepaic_evs), dv = dv + '_scaled'))

In [ ]:
Stargazer(rating_models)

In [ ]:
print(Stargazer(rating_models).render_latex())

In [ ]:
company_stocks.columns

In [ ]:
evs = ["physiological", "psychological", 'total reviews']

for ev in evs:
    company_stocks[ev + '_scaled'] = scale_var(company_stocks, ev)
    
evs = [ev + '_scaled' for ev in evs]    

dvs = ['rating_balance',
       'rating_career', 'rating_culture', 'rating_mgmt', 'rating_comp', 
      # 'rating_overall'
      ]

In [ ]:
# with the differently aggregated data

rating_models = []
for dv in dvs:
    company_stocks[
        dv + '_scaled'] = scale_var(company_stocks, dv)
    rating_models.append(regress(company_stocks, evs = evs, dv = dv + '_scaled'))
    
# with stepaic

rating_models = []
for dv in dvs:
    stepaic_evs, _ = stepaic_model(company_stocks, evs, dv + '_scaled')
    rating_models.append(regress(company_stocks, evs = list(stepaic_evs), dv = dv + '_scaled'))    

In [ ]:
dvs

In [ ]:
evs

In [ ]:
# in paper (Table 4)
Stargazer(rating_models)

In [ ]:
print(Stargazer(rating_models).render_latex())

In [ ]:
# predict ratings for ALL 104 companies and not just those that have stock data
all_companies = company_data[text][1].copy()

In [ ]:
all_companies = all_companies.merge(company_total_reviews, on = 'company_name')
all_companies.head()

In [ ]:
rating_cols = ['rating_balance',
 'rating_career',
 'rating_comp',
 'rating_culture',
 'rating_mgmt',
 'rating_overall']

In [ ]:
# get the ratings

onek_data_2019_company = onek_data.groupby('company_name').mean()[rating_cols]
onek_data_2019_company = onek_data_2019_company.reset_index()
onek_data_2019_company.columns = ['company_name', 'rating_balance',
 'rating_career',
 'rating_comp',
 'rating_culture',
 'rating_mgmt',
 'rating_overall'
                                 ]

all_companies = all_companies.merge(onek_data_2019_company, on = 'company_name')
len(all_companies)

In [ ]:
evs = ['composite', 
#       'physiological',
#       'psychological',
      'total reviews'
      ]

for ev in evs:
    all_companies[ev + '_scaled'] = scale_var(all_companies, ev)
    
evs = [ev + '_scaled' for ev in evs]    

dvs = ['rating_balance',
       'rating_career', 'rating_culture', 'rating_mgmt', 'rating_comp', 
      # 'rating_overall'
      ]

# with the differently aggregated data

rating_models = []
for dv in dvs:
    all_companies[dv + '_scaled'] = scale_var(all_companies, dv)
    rating_models.append(regress(all_companies, evs = evs, dv = dv + '_scaled'))
    
# with stepaic

rating_models = []
for dv in dvs:
    stepaic_evs, _ = stepaic_model(all_companies, evs, dv + '_scaled')
    rating_models.append(regress(all_companies, evs = evs, dv = dv + '_scaled'))    

In [ ]:
len(all_companies)

In [ ]:
stepaic_evs, dvs, evs

In [ ]:
Stargazer(rating_models)

In [ ]:
print(Stargazer(rating_models).render_latex())

### Predict ODI / boredom from the ratings

In [ ]:
dvs = ['physiological', 'psychological', 'composite']

evs = ['rating_balance',
       'rating_career', 'rating_culture', 'rating_mgmt', 'rating_comp', 
       'total reviews'
      # 'rating_overall'
      ]


for ev in evs:
    all_companies[ev + '_scaled'] = scale_var(all_companies, ev)
    
evs = [ev + '_scaled' for ev in evs]    


# with the differently aggregated data

rating_models = []
for dv in dvs:
    all_companies[dv + '_scaled'] = scale_var(all_companies, dv)
    rating_models.append(regress(all_companies, evs = evs, dv = dv + '_scaled'))
    
# with stepaic

rating_models = []
for dv in dvs:
    stepaic_evs, _ = stepaic_model(all_companies, evs, dv + '_scaled')
    rating_models.append(regress(all_companies, evs = list(stepaic_evs), dv = dv + '_scaled'))    

In [ ]:
all_companies.corr()

In [ ]:
Stargazer(rating_models)

In [ ]:
print(Stargazer(rating_models).render_latex())

# state level

In [ ]:
# load external data
# load dependent variable and control variables
states = pd.read_csv("../data/us_states_geo.csv")
states['State Abbreviation'] = states['state_code']
states = states[['State Abbreviation', 'population_2019']]

In [ ]:
mega_state_data = pd.read_csv(DATAROOT + "US_States/US_State_Control_Data_Master.csv", sep = "\t")
mega_state_data = mega_state_data.dropna(axis=1, how='all')

In [ ]:
solar = pd.read_csv(DATAROOT + "US_States/US States Solar Power.csv")
mega_state_data = mega_state_data.merge(solar[['State Abbreviation', 'SEIA Ranking 2017']],
                                        on = 'State Abbreviation')

In [ ]:
creative_class = pd.read_csv(DATAROOT + "US_States/creative_class.csv")
creative_class['State'] = creative_class['State ']
extra_spaces = list(set(creative_class['State'].values) - set(mega_state_data['State'].values))
creative_class['State'] = [i[:-1] if i in extra_spaces else i for i in creative_class['State']]
creative_class['State'] = [i[1:] if i == ' Massachusetts' else i for i in creative_class['State']]

In [ ]:
mega_state_data = mega_state_data.merge(creative_class[['State', 'Creativity Index ']], on = 'State')

In [ ]:
# what about depression

mega_state_data['H_2a_depression_num'] = [float(i[:-1]) for i in mega_state_data['H_2a_depression']]

In [ ]:
personality = pd.read_csv(DATAROOT + "US_States/personality.csv", sep = "\t")
personality = personality.dropna(axis=1, how='all')
personality.columns = ['State', 'Sample size', 'Extraversion', 'Agreebleness',
                       'Conscientiousness', 'Neuroticism', 'Openness']

mega_state_data = mega_state_data.merge(personality[['State', 'Extraversion', 'Agreebleness',
                       'Conscientiousness', 'Neuroticism', 'Openness']], on = 'State')

In [ ]:
states = states.merge(mega_state_data, on = 'State Abbreviation')
states['population_2019_log'] = np.log(states['population_2019'])
states

In [ ]:
selected_columns = []

# only keep numerical variables

for col in states.columns:
    if states[col].dtype in numerics:
        selected_columns.append(col)

In [ ]:
# dv's: 'E_2a_gdp' and 'P_3c_giniCoefficient'
for i in ['E_2a_gdp', 'P_3c_giniCoefficient',
          'population_2019', 'FIPS Code', 'Alt FIPS Code', 
          #'State',
          'H_2a_depression_num',
          'S_7b_trustGeneral' # this one has lots of NaNs
         ]:
    try:
        selected_columns.remove(i)    
    except Exception as e:
        print(e)
        print(i)
        pass   

In [ ]:
from utils import aggregate_area

# def aggregate_area(data, final_goals, onek_data, sustainability,
#                    metric = 'avg sim score',
#                    area_name = 'company_name', scaled = False):


In [ ]:
# add sustainability variables
state_data = {}


for text in [text]:
    data_90 = subset_by_percentile_or_preset(main_datas_[text], THRESHOLD, preset = PRESET, num = num_goals)
    if sent_based:
        data_90 = aggregate_sents(data_90, review, num_goals = num_goals)              
    state_data[text] = aggregate_area(data_90, final_goals, onek_data,
                                      sustainability, area_name = 'state',
                                      construct = construct,
                                      find_pcs = False,
                                      reference_text = reference_text,
                                      scaled = True)

state_columns = list(state_data[text][1].columns)
state_columns[0] = "State Abbreviation"
state_data[text][1].columns = state_columns
states = states.merge(state_data[text][1], on = "State Abbreviation")

In [ ]:
df = state_data[text][1]

In [ ]:
df

In [ ]:
states[['composite', 'population_2019', 
        'Openness', 'E_2a_gdp',
        'Creativity Index ', 'H_2a_depression_num']].corr()

In [ ]:
if split:
    states[['psychological', 'physiological', 'composite', 'Openness', 'E_2a_gdp', 'H_2a_depression_num']].corr()

In [ ]:
state_total_reviews = pd.DataFrame(review.groupby('state').size()).reset_index()
state_total_reviews.columns = ['State Abbreviation', 'total reviews']
states = states.merge(state_total_reviews, on = 'State Abbreviation')
states['total reviews_log'] = np.log(states['total reviews'])
# interaction
states['total_reviews_state_population_interaction'] = states['total reviews_log'] * states['population_2019_log']
states = states.dropna(subset = ['P_3c_giniCoefficient', 'E_2a_gdp'])
states

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 6, figsize = (24, 4), sharey = True)
for n, num in enumerate(final_goals):
    sns.scatterplot(data = states, y = 'E_2a_gdp', x = '%d %s' %(num, metric), ax = ax[n])
    corr, p = stats.pearsonr(states['E_2a_gdp'], states['%d %s' %(num, metric)])
    ax[n].set_title("%s\n%0.2f %0.2f" %(primary_goal_shorthand[sustainability[reference_text][num]], corr, p), fontsize = 18)
    if n == 0:
        ax[n].set_ylabel('E_2a_gdp', fontsize = 18)
    ax[n].set_xlabel(metric, fontsize = 18)
    

In [ ]:
states['E_2a_gdp_log'] = np.log(states['E_2a_gdp'])
fig, ax = plt.subplots(nrows = 1, ncols = 6, figsize = (24, 4), sharey = True)
for n, num in enumerate(final_goals):
    sns.scatterplot(data = states, y = 'E_2a_gdp_log', x = '%d %s' %(num, metric), ax = ax[n])
    corr, p = stats.pearsonr(states['E_2a_gdp_log'], states['%d %s' %(num, metric)])
    ax[n].set_title("%s\n%0.2f %0.2f" %(primary_goal_shorthand[sustainability[reference_text][num]], corr, p), fontsize = 18)
    if n == 0:
        ax[n].set_ylabel('E_2a_gdp_log', fontsize = 18)
    ax[n].set_xlabel(metric, fontsize = 18)
    

In [ ]:
states['P_3c_giniCoefficient_log'] = np.log(states['P_3c_giniCoefficient'])
fig, ax = plt.subplots(nrows = 1, ncols = 6, figsize = (24, 4), sharey = True)
for n, num in enumerate(final_goals):
    sns.scatterplot(data = states, y = 'P_3c_giniCoefficient_log', x = '%d %s' %(num, metric), ax = ax[n])
    corr, p = stats.pearsonr(states['P_3c_giniCoefficient_log'], states['%d %s' %(num, metric)])
    ax[n].set_title("%s\n%0.2f %0.2f" %(primary_goal_shorthand[sustainability[reference_text][num]], corr, p), fontsize = 18)
    if n == 0:
        ax[n].set_ylabel('P_3c_giniCoefficient_log', fontsize = 18)
    ax[n].set_xlabel(metric, fontsize = 18)
    

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 6, figsize = (24, 4), sharey = True)
for n, num in enumerate(final_goals):
    sns.scatterplot(data = states, y = 'population_2019_log', x = '%d %s' %(num, metric), ax = ax[n])
    corr, p = stats.pearsonr(states['population_2019_log'], states['%d %s' %(num, metric)])
    ax[n].set_title("%s\n%0.2f %0.2f" %(primary_goal_shorthand[sustainability[reference_text][num]], corr, p))

In [ ]:
sns.scatterplot(data = states, x = 'E_2a_gdp', y = 'total reviews_log')

## Predicting GDP, Gini, and Creativity

- controls: urban population
- dvs: gdp, gini, creativity (all logged and scaled)
- 5 types of models for each:
    - control only
    - control + composite sustainability var [FAIR]
    - control + all sustainability vars
    - stepAIC(control + sustainability vars) ---> pick the best
    - control + best from previous step [FAIR]  

In [ ]:
sustainability_vars = ['%s' %(primary_goal_shorthand[sustainability[reference_text][i]]) for i in final_goals]
new_names = dict([('%d %s' %(i, metric), '%s' %primary_goal_shorthand[sustainability[reference_text][i]]) for i in final_goals])
states = states.rename(new_names, axis = 1)

In [ ]:
# scale the predictors (urban pop, composite score)

# log every numerical column

for col in states.columns:
    if states[col].dtype in numerics:
        states[col + '_log'] = np.log(states[col])


var_list = ['S_9a_urbanPop_log', 'Openness_log', 'E_2a_gdp_log',
            'Creativity Index ', 'H_2a_depression_num_log', 'composite']
            #, 'rank-diff', 'PCA2']
    # creativity is not skewed so no need to log it

for var in var_list:
    states[var + ' scaled'] = scale_var(states, var)

In [ ]:
states[[i + ' scaled' for i in var_list]].head()

In [ ]:
def multi_model_regress(data, target, controls, composite, sustainable_vars, best):
    # model 1
    evs1 = controls
    # model 2
    evs2 = controls + composite
    # model 3
    evs3 = controls + sustainability_vars
    # model 4
    evs = controls + sustainability_vars
    stepaic_vars, aic = stepaic_model(data, evs, target)
    evs4 = list(set(list(stepaic_vars) + controls))
    # model 5
    evs5 = controls + best
    
    model1 = regress(data, evs1, target)
    model2 = regress(data, evs2, target)
    model3 = regress(data, evs3, target)
    model4 = regress(data, evs4, target)
    model5 = regress(data, evs5, target)
    
    #print(evs1, evs2, evs3, evs4)
    return model1, model2, model3, model4, model5

In [ ]:
# reverse inequlaity to equality

# states['P_3c_giniCoefficient_log scaled'] = 1 - states['P_3c_giniCoefficient_log scaled']

In [ ]:
gdp = 'E_2a_gdp_log scaled'
gini = 'Openness_log scaled'
creativity = 'Creativity Index  scaled'
depression = 'H_2a_depression_num_log scaled'


sustainability_score = 'composite scaled'

In [ ]:
# no need to log or scale or scale for plots

gdp_ = 'E_2a_gdp'
gini_ = 'Openness'
creativity_ = 'Creativity Index '
depression_ = 'H_2a_depression_num'


sustainability_score_ = 'composite'

In [ ]:
cols = ['S_9a_urbanPop', 'Openness', 'E_2a_gdp',
            'Creativity Index ', 'H_2a_depression_num', 'composite']
states[cols].corr()

In [ ]:
sns.distplot(states['Creativity Index '])

In [ ]:
stats.pearsonr(states['composite'], states['Creativity Index _log'])

In [ ]:
len(states)

In [ ]:
controls = ['S_9a_urbanPop_log scaled']
null_gdp, composite_gdp, all_gdp, stepaic_gdp, best_gdp = multi_model_regress(states, gdp,
                                                                    controls, [sustainability_score], 
                                                                    sustainability_vars,
                                                                    best = [sustainability_score])
Stargazer([null_gdp, composite_gdp, best_gdp])

In [ ]:
states.columns

In [ ]:
null_gini, composite_gini, all_gini, stepaic_gini, best_gini = multi_model_regress(states, gini,
                                                                    controls, [sustainability_score], 
                                                                    sustainability_vars,
                                                                    best = [sustainability_score])
Stargazer([null_gini, composite_gini, best_gini])

In [ ]:
null_cc, composite_cc, all_cc, stepaic_cc, best_cc = multi_model_regress(states, creativity,
                                                                    controls, [sustainability_score], 
                                                                    sustainability_vars,
                                                                    best = [sustainability_score])
Stargazer([null_cc, composite_cc, all_cc, stepaic_cc, best_cc])

In [ ]:
null_d, composite_d, all_d, stepaic_d, best_d = multi_model_regress(states, depression,
                                                                    controls, [sustainability_score], 
                                                                    sustainability_vars,
                                                                    best = [sustainability_score])
Stargazer([null_d, composite_d, all_d, stepaic_d, best_d])

In [ ]:
Stargazer([null_gdp, null_gini, null_cc, null_d])

In [ ]:
Stargazer([composite_gdp, composite_gini, composite_cc, composite_d])

In [ ]:
print(Stargazer([composite_gdp, best_gdp, composite_gini, best_gini, composite_cc, best_cc]).render_latex())

In [ ]:
from adjustText import adjust_text
def label_point(x, y, val, ax, adjust =  False, gap = .01):
    texts = []
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        texts.append(ax.text(point['x']+gap, point['y'], str(point['val']), fontsize = 9))
    if adjust:
        adjust_text(texts, ax = ax)

def sanity_plot(data, var_pairs, nrows, ncols, limit = 0, labeled = False):
    if limit == 0:
        limit = nrows * ncols
    num = 0
    fig, axs = plt.subplots(nrows = nrows, ncols = ncols, figsize = (5*ncols, 5*nrows), sharey = True)
    for i in range(nrows):
        for j in range(ncols):
            x, y = var_pairs[num]
            sns.regplot(data = data, y = y, x = x, ax = axs[i][j])
            corr, p = stats.pearsonr(data[y], data[x])
            axs[i][j].set_title('R = %0.2f (p = %0.3f)' %(corr, p), fontsize = 16)
            if labeled:
                label_point(data[x], data[y],
                        data['State Abbreviation'], axs[i][j])
            num += 1
            if num == limit:
                break
        if num == limit:
            break
    plt.tight_layout()
    if labeled:
        title = "regressions/sanity_check_regressions_labeled.pdf"
    else:
        title = "regressions/sanity_check_regressions.pdf"
    #plt.savefig(PLOTROOT + title)

In [ ]:
def row_plots(data, var_pairs, labeled = False):
    fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (15, 7), sharey = True)
    for num in range(2):
        x, x_name, y, y_name = var_pairs[num]
        sns.regplot(data = data, y = y, x = x, ax = axs[num])
        corr, p = stats.pearsonr(data[y], data[x])
        axs[num].set_title('r = %0.2f (p = %0.3f)' %(corr, p), fontsize = 16)
        #if num == 0:
        axs[num].set_ylabel(y_name)
#         else:
#             axs[num].set_ylabel('')
        axs[num].set_xlabel(x_name)
        if labeled:
            label_point(data[x], data[y], data['State Abbreviation'], axs[num], adjust = True)
    plt.tight_layout()
    if labeled:
        title = "%s_regressions_labeled.pdf" %y
    else:
        title = "%s_regressions.pdf" %y
    #plt.savefig(PLOTROOT + title)

In [ ]:
def row_col_triplet_plots(data, var_pairs, labeled = False, ncols = 2, nrows = 2):
    fig, axs = plt.subplots(nrows = nrows, ncols = ncols, figsize = (5*ncols, 5*nrows), sharey = 'row')
    num = 0
    for row in range(nrows):
        for col in range(ncols):
            x, x_name, y, y_name = var_pairs[num]
            corr, p = stats.pearsonr(data[y], data[x])
            if p > 0.05:
                sns.regplot(data = data, y = y, x = x, ax = axs[row][col], color = 'grey')
            else:
                sns.regplot(data = data, y = y, x = x, ax = axs[row][col])
            axs[row][col].set_title('r = %0.2f (p = %0.3f)' %(corr, p), fontsize = 16)
            #if col == 0:
            axs[row][col].set_ylabel(y_name)
            #else:
            #    axs[row][col].set_ylabel('')
            axs[row][col].set_xlabel(x_name)
            if labeled:
                label_point(data[x], data[y], data['State Abbreviation'], axs[row][col], adjust = True)
            num += 1
            
            
    if labeled:
        title = "wealth_inequality_cc_emergent_regressions_labeled_triplet.pdf"
    else:
        title = "wealth_inequality_cc_emergent_regressions_triplet.pdf"
        
        
    plt.tight_layout()
    plt.savefig(PLOTROOT + title)

In [ ]:
states['H_2a_depression_num'].head()

In [ ]:
var_pairs = [(sustainability_score_, "composite", gdp_, "wealth"),
             ("physiological", "physiological ODI", gdp_, "wealth"),
             ("psychological", "psychological ODI", gdp_, "wealth"),
             (sustainability_score_, "composite", gini_, "equality"),
             ("physiological", "physiological ODI", gini_, "equality"),
             ("psychological", "psychological ODI", gini_, "equality"),
             (sustainability_score_, "composite", creativity_, "creativity"),
             ("physiological", "physiological ODI", creativity_, "creativity"),
             ("psychological", "psychological ODI", creativity_, "creativity"),
             (sustainability_score_, "composite", 'H_2a_depression_num', "depression"),
             ("physiological", "physiological ODI", 'H_2a_depression_num', "depression"),
             ("psychological", "psychological ODI", 'H_2a_depression_num', "depression"),
            ]
row_col_triplet_plots(states, var_pairs, ncols = 3, nrows = 4, 
                     # labeled = True
                     )

In [ ]:
def single_row_plots(data, var_pairs, labeled = False, ncols = 4, nrows = 1):
    fig, axs = plt.subplots(nrows = nrows, ncols = ncols, figsize = (5*ncols, 5*nrows), sharey = False)
    num = 0
    for col in range(ncols):
        x, x_name, y, y_name = var_pairs[num]
        corr, p = stats.pearsonr(data[y], data[x])
        if p > 0.05:
            sns.regplot(data = data, y = y, x = x, ax = axs[col], color = 'grey')
        else:
            sns.regplot(data = data, y = y, x = x, ax = axs[col])
        axs[col].set_title('R = %0.2f (p = %0.3f)' %(corr, p), fontsize = 22)
        #if col == 0:
        if col == 3:
            num_is = "iv"
        else:
            num_is = "i" * (col+1)
        axs[col].set_ylabel("%s) %s" %(num_is, y_name), fontsize = 22)
        axs[col].set_xlabel(x_name)
        if labeled:
            label_point(data[x], data[y], data['State Abbreviation'], axs[col], adjust = True)
        num += 1
            
            
    if labeled:
        title = "depression_wealth_cc_openness_regressions_labeled.pdf"
    else:
        title = "depression_wealth_cc_openness_regressions_labeled.pdf"
        
        
    plt.tight_layout()
    plt.savefig(PLOTROOT + title)

In [ ]:
def reformat_large_tick_values(tick_val, pos):
    """
    Turns large tick values (in the billions, millions and thousands) such as 4500 into 4.5K and also appropriately turns 4000 into 4K (no zero after the decimal).
    """
    if tick_val >= 1000000000:
        val = round(tick_val/1000000000, 1)
        new_tick_format = '{:}B'.format(val)
    elif tick_val >= 1000000:
        val = round(tick_val/1000000, 1)
        new_tick_format = '{:}M'.format(val)
    elif tick_val >= 1000:
        val = round(tick_val/1000, 1)
        new_tick_format = '{:}K'.format(val)
    elif tick_val < 1000:
        new_tick_format = round(tick_val, 1)
    else:
        new_tick_format = tick_val

    # make new_tick_format into a string value
    new_tick_format = str(new_tick_format)
    
    # code below will keep 4.5M as is but change values such as 4.0M to 4M since that zero after the decimal isn't needed
    index_of_decimal = new_tick_format.find(".")
    
    if index_of_decimal != -1:
        value_after_decimal = new_tick_format[index_of_decimal+1]
        if value_after_decimal == "0":
            # remove the 0 after the decimal point since it's not needed
            new_tick_format = new_tick_format[0:index_of_decimal] + new_tick_format[index_of_decimal+2:]
            
    return new_tick_format

In [ ]:
def single_plots(data, var_pairs, labeled = False, ncols = 4, nrows = 1):
    import matplotlib.ticker as tkr

    formatter = tkr.ScalarFormatter(useMathText=True)
    formatter.set_scientific(True)
        #formatter.set_powerlimits((-2, 2))
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (6, 4.5), sharey = False)
    num = 0
    for col in range(ncols):
        x, x_name, y, y_name = var_pairs[num]
        corr, p = stats.pearsonr(data[y], data[x])
        if p > 0.05:
            ax = sns.regplot(data = data, y = y, x = x, color = 'grey')
        else:
            ax = sns.regplot(data = data, y = y, x = x, ax = ax, line_kws={"format": formatter})
        ax.set_title('R = %0.2f (p = %0.3f)' %(corr, p), fontsize = 24, fontweight = 'bold')
        #if col == 0:
        if col == 3:
            num_is = "iv"
        else:
            num_is = "i" * (col+1)
        ax.set_ylabel(y_name, fontsize = 24, fontweight = 'bold')
        ax.set_xlabel(x_name, fontsize = 24, fontweight = 'bold')
        ax.set_yticklabels(ax.get_yticks(), size = 16)
        ax.set_xticklabels(ax.get_xticks(), size = 16)
        ax.set_yticklabels(ax.get_yticks().astype(int))
        ax.set_xticklabels(ax.get_xticks().astype(int))
        extra = data[y].quantile(.01)
        ax.set_ylim(min(data[y]) - extra, max(data[y]) + extra)

        
        
        
        if labeled:
            label_point(data[x], data[y], data['State Abbreviation'], ax, adjust = True)
        num += 1
        
        title = y_name + ".pdf"
        
        plt.tight_layout()
        plt.savefig(PLOTROOT + title)     
        plt.cla()
    

In [ ]:
var_pairs = [(sustainability_score, "ODI", 'H_2a_depression_num', "Depression"),
             (sustainability_score, "ODI", creativity_, "Creativity"),
             (sustainability_score, "ODI", gini_, "Openness"),
             (sustainability_score, "ODI", gdp_, "Wealth"),
            ]
single_plots(states, var_pairs, ncols = 4, nrows = 1, 
                     #labeled = True
                     )

In [ ]:
sustainability_vars

In [ ]:
# predict wealth, inequality, creativity with PCA vars (PC1, PCA2, rank-diff)

In [ ]:
def multi_model_regress(data, target, controls, composite, independent_vars, best):
    # model 1
    evs1 = controls
    # model 2
    evs2 = controls + composite
    # model 3
    evs3 = controls + independent_vars
    # model 4
    evs = controls + independent_vars
    stepaic_vars, aic = stepaic_model(data, evs, target)
    #evs4 = list(set(list(stepaic_vars) + controls))
    evs4 = list(stepaic_vars)
    # model 5
    evs5 = controls + best
    
    model1 = regress(data, evs1, target)
    model2 = regress(data, evs2, target)
    model3 = regress(data, evs3, target)
    model4 = regress(data, evs4, target)
    model5 = regress(data, evs5, target)
    
    #print(evs1, evs2, evs3, evs4)
    return model1, model2, model3, model4, model5

In [ ]:
controls = ['S_9a_urbanPop_log scaled']

if split:
    independent_vars = ['physiological', 'psychological']
    states['physiological x psychological'] = states['physiological'] * states['psychological']
    interactions = ['physiological x psychological']
else:
    independent_vars = ['composite']
    interactions = []
    
# if True:
#     independent_vars = ['composite']
#     interactions = []

# scale the indepenedent variables

for var in independent_vars:
    states[var] = scale_var(states, var)

In [ ]:
# add interactions if needed
for var in independent_vars:
    states['%s x urban' %var] = states[var] * states['S_9a_urbanPop_log scaled']
    interactions.append('%s x urban' %var)
    
for var in interactions:
    states[var] = scale_var(states, var)    
    
# add some interactions
if interact:
    independent_vars = independent_vars + interactions
    
        

In [ ]:
interact, independent_vars

In [ ]:
null_gdp, composite_gdp, all_gdp, stepaic_gdp, best_gdp = multi_model_regress(states, gdp,
                                                                    controls, independent_vars, 
                                                                    independent_vars,
                                                                    best = independent_vars)

null_gini, composite_gini, all_gini, stepaic_gini, best_gini = multi_model_regress(states, gini,
                                                                    controls, independent_vars, 
                                                                    independent_vars,
                                                                    best = independent_vars)

null_cc, composite_cc, all_cc, stepaic_cc, best_cc = multi_model_regress(states, creativity,
                                                                    controls, independent_vars, 
                                                                    independent_vars,
                                                                    best = independent_vars)

null_d, composite_d, all_d, stepaic_d, best_d = multi_model_regress(states, depression,
                                                                    controls, independent_vars, 
                                                                    independent_vars,
                                                                    best = independent_vars)


Stargazer([composite_gdp, stepaic_gdp, composite_gini, stepaic_gini, composite_cc, stepaic_cc])

In [ ]:
Stargazer([composite_gdp, best_gdp, composite_gini, best_gini, composite_cc, best_cc])

In [ ]:
# in paper, Table 5
Stargazer([stepaic_d, stepaic_gdp, stepaic_cc, stepaic_gini])

In [ ]:
print(Stargazer([stepaic_d, stepaic_gdp, stepaic_cc, stepaic_gini]).render_latex())

In [ ]:
# in paper, Table 5
Stargazer([all_d, all_gdp, all_cc, all_gini])

In [ ]:
print(Stargazer([all_d, all_gdp, all_cc, all_gini]).render_latex())

In [ ]:
# in paper, Table 5
Stargazer([null_d, null_gdp, null_cc, null_gini])

In [ ]:
len(states)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

In [ ]:
list(states.columns)

In [ ]:
states['ODI'] = states['composite']
states['Urban Population'] = states['S_9a_urbanPop']
states['Depression'] = states['H_2a_depression_num']
states['Wealth'] = states['E_2a_gdp']
states['Creativity'] = states['Creativity Index ']

for var in ['Openness',
                                                               'Extraversion',
                                                               'Agreebleness',
                                                               'Conscientiousness',
                                                               'Neuroticism']:
    states['%s' %str.upper(var[0])] = states[var]

In [ ]:
cross_corr_vars = ['ODI',
                   'Urban Population'] + ['Depression',
                                          'Wealth',
                                          'Creativity'] + ['O', 'C', 'E', 'A', 'N']
                                                               
                                    

rows = []
for i in range(0, len(cross_corr_vars)):
    var1 = cross_corr_vars[i]
    row = []
    for j in range(0, i+1):
        var2 = cross_corr_vars[j]
        corr, p = stats.pearsonr(states[var1], states[var2])
        if var1 == var2:
            row.append("---")
            continue
        if p < 0.005:
            stars = "***"
        elif p < 0.01:
            stars = "**"
        elif p < 0.05:
            stars = "*"
        else:
            stars = ""
        row.append(str("%0.3f" %corr)+stars)
    rows.append(row)
    
cross_corr = pd.DataFrame(rows, columns = cross_corr_vars)    
cross_corr["var"] = cross_corr_vars
cross_corr = cross_corr.set_index("var")
cross_corr = cross_corr.fillna(" ")
cross_corr

In [ ]:
print(cross_corr.to_latex())

In [ ]:
states[['Depressed mood',
 'Sleep alterations',
 'Fatigue/Loss of energy',
 'Feelings of worthlessness',
 'composite']].corr()

In [ ]:
company_stocks[['1 avg sim score',
 '2 avg sim score',
 '3 avg sim score',
 '5 avg sim score',
 'composite']].corr()

In [ ]:
for var in cross_corr_vars:
    print(var)
    ax = sns.distplot(states[var], kde = False, color ='black', hist_kws=dict(alpha=1))
    plt.axis('off')
    #plt.show()
    plt.savefig(PLOTROOT + "distplots/%s.png" %var)
    plt.cla()

In [ ]:
len(states)

In [ ]:
states[cross_corr_vars].describe().T

In [ ]:
states[cross_corr_vars].describe().T[['min', 'max', '50%', 'mean', 'std']]

In [ ]:
print(states[cross_corr_vars].describe().T[['min', 'max', '50%', 'mean', 'std']].round(2).to_latex())

In [ ]:
states['composite']

In [ ]:
company